##1. Specific Aim

For the final project I will be applying natural language processing (NLP) techniques to cluster Hillary Clinton's State Department emails into distinct topics or categories. For example, an email specifically discussing the controversial raid on Benghazi would be clustered with similar emails. 

   ### Data Source
    
The data used for this project are, former Secretary of State, Hillary Clinton's State Department emails. The emails were originally releases in PDF format but were scraped and posted on Kaggle on September 11, 2015. 

The data are stored in a SQLite database containing five tables:

In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect('input/database.sqlite')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print [x for x in cursor.fetchall()]

[(u'Emails',), (u'Persons',), (u'Aliases',), (u'EmailReceivers',)]


1. Emails
  1. This dataframe contains all relevant email information including the email subject, date sent, sender name, and email body.
2. Persons
  1. This dataframe contains a list of all the sender names with a unique identifier to merge on other tables
3. Aliases
  1. This dataframe contains a list of sender aliases that correspond to a unique person in the Persons table. 
4. EmailReceivers
  1. This dataframe links an unique email ID with a sender person ID and alias ID.

For the purposes of this project, a majority of the analysis will be performed on the **Emails** table which contains the body text of the email.

###Sample Size

The emails datasets contains 7,945 emails. However, a portion of these emails are heavily redacted and contain no remaining email text. After we remove emails without text, we are left with 6,742 emails. 

In [13]:
emails_all= pd.read_sql_query("Select * From Emails",con)
emails= pd.read_sql_query("Select * From Emails where ExtractedBodyText!=''",con)
print emails_all.shape
print emails.shape

(7945, 22)
(6742, 22)


###Time Period
The emails begin in 2009 and end in 2012.

###Preliminary Hypothesis 
The analysis I am planning is entirely unsupervised and, as a result, it does lend itself to traditional hypotheses a priori. My plan is to apply the following topic modeling techniques to Hillary Clinton's emails and evaluate/compare the performance:
  1. K-Means clustering and
  2. Latent-Dirichlet Allocation (**LDA**)
  
I am not concerned with prediction and would classify the project as an exploration of various clustering/topic modeling techinques rather than a traditional train/test model. Overall, the goal is to understand which of these techniques performs the "best" in defining a cluster or topic for a specific email. I put "best" in quotes because determining whether a specific topic is appropriate to an email is subjective to my interpretation. However, moving forward I am looking to identify different ways to classify "best" in this context.

One obvious approach would be to apply some cross-validation techniques to assess the performance of each technique on out-of-sample data. My one concern with this approach is that it would diminish my already small sample size of emails (6,742).

Another possible evalution approach, specific to LDA only, would be to put the words associated with each topic through **Word2Vec** in gensim. Then calculate similarity measures between words in a topic to determine whether the topic defined by LDA was "coherent".

A final approach for evaluation would be performing _word intrusion_ as defined by [Chang et. al](http://www.umiacs.umd.edu/~jbg/docs/nips2009-rtl.pdf). They describe a method where for each trained topic, a word is chosen at random and then substituted into the first ten (or five) word associated with a topic. Then if a human is able to detect which word is not appropriate for that topic it is said that the topic is coherent. My approach would try to take advantage of the **doesnt_match** function in **Word2Vec**, rather than using human judgement. 

Overall, my expectation is that rather than one clearly superior method, a combination of clustering/topic modeling techniques will need to be used to create the best classifier. For example, it may be the case that LDA is better at identifying emails concerned with _Yoga_ and K-Means is better at identifying emails concerned with _Libya_.
  


##2. Methods

###Outcome
As mentioned above, this is an unsupervised learning approach with no established outcome a priori. None of the emails are categorized into topics before they are run through each of the categorization or topic modeling algorithms.

###Predictors
The predictor used for each algorithm is some numerical representation of the words in each email, either a **term frequency - inverse document frequency matrix** or a **document-term matrix** . At the beginning, however, each email body is a long string of characters (see below). Notice that each email is formatted differently and in some cases the .PDF scraping was not perfect so there are some extraneous words.


In [3]:
print "Example Email #1:"
print "'",emails.ExtractedBodyText[789],"'"
print " "
print "Example Email #2:"
print "'",emails.ExtractedBodyText[1234],"'"

Example Email #1:
' What is his email at White House? '
 
Example Email #2:
' Secretary Clinton,
I hope that all is well back in the States and just wanted to let you know that I will be in Washington
Wednesday. If you might have a minute to spare to follow up on your last message in person or by phone this
week or over the weekend, I would very much appreciate the opportunity. Congratulations on the Europe
trip!
Yours sincerely, Jackie
Jacqueline Newmyer
President, Long Term Strategy Group
12 Eliot St., Cambridge, MA 02138
617-661-1626 (fax)
www.ltstrategy.com '


The body of each email will need to cleaned and tokenized in order to run through the various algorithms. At the moment I am still determing whether stemming is a necessary part of the cleaning process. By stemming, I mean reducing words to their root and eliminating any suffixes or prefixes. From my initial work, stemming seems to return strange word results. For example the stemmed version of "secretary" is "secretari" (see below) or "house" is "hous".

In [4]:
%run ./review_to_words.py

In [5]:
print "Example Email #1: Cleaned"
print review_to_words(emails["ExtractedBodyText"][789])
print " "
print "Example Email #2: Cleaned"
print review_to_words(emails["ExtractedBodyText"][1234])

Example Email #1: Cleaned
email white hous
 
Example Email #2: Cleaned
secretari clinton hope well back state want let know washington wednesday might minut spare follow last messag person phone week weekend would much appreci opportun congratul europ trip sincer jacki jacquelin newmyer presid long term strategi group eliot st cambridg ma fax www ltstrategi com


###Algorithms under consideration

I plan to explore the following clustering and topic modeling techniques:
  1. K-Means clustering and,
  2. Latent-Dirichlet Allocation (**LDA**),
  
####K-Means Clustering
When it comes to unsupervised text analysis and grouping documents or texts together, K-Means Clustering seems to be a very popular approach. Also, since it was one of the very first techniques we learned, it seemed to be a good place to start the exploration of text clustering. 

To perform the K-Means clustering, each email will need to be transformed into a numerical dataset that the K-Means algorithm can interpret. (Of course, of all this will have to be performed after significant text cleaning.) To accomplish this, all the emails will be represented in a term frequency - inverse document frequency (**tf-idf**) matrix. From my understanding, each column in this tf-idf matrix will represent an email and each row will represent a word. The value for a specific word and email combination is the term frequency multiplied by the inverse document frequency. The term frequency is simply the ratio of the number of occurrences of that word over the total word length of the email. The inverse document frequency is the log of ratio of the total number of emails over the total number of emails containing that word. By multiplying these two values together the tf-idf value assigns greater weight to words that appear frequently in the email but are rare across all emails.

Once the words are represented in a numerical dataframe, I will use the  **scikit** **kmeans( )** algorithm to determine the clustering of each email. Note that this algorithm will return a distinct cluster group for each email, or a distinct topic for each email. This is not the case in LDA. 

####Latent-Dirichlet Allocation

The second technique I will be using is Latent-Dirichlet Allocation (LDA). I chose LDA because it was consistently referenced as the most popular technique for topic modeling. Latent Dirichlet is a unsupervised learning technique that defines topics from a set of texts. The number of topics the LDA algorithm defines is set by the user.

From my very basic understanding of LDA, to define topics the algorithm makes assumptions on how a document or a piece of text was made and then reverses the process to backout a particular set of topics. LDA assumes that documents, or in our case emails, are written in following steps:

1. Determine the number of words the email will have according to some distribution.
2. Determine a mixture of K topics to be used in the email. For example, the email may be split between 'Benghazi'(50%),'House Committee' (30%), and 'Election" (20%). These three topics also have their own distribution of words. For example:
  * Benghazi: Qaddaffi (50%) and Libya(50%)
  * House Committee: Boehner (50%) and Investigation(50%)
  * Election: Polls (50%) and Vote(50%)
3. Then to pick each word in the email you:
  1. Pick a topic according to the distribution 'Benghazi'(50%),'House Committee' (30%), and 'Election" (20%).
  2. Pick a word from the topic distribution.
  
The LDA algorithm 'reverses' this process to backout the topic distribution for each email. The algorithm begins by randomly assigning words in each email to a topic. This, of course, assigns topics to words that do not make any sense. The algorithm then reassigns each word a new topic based on the probablity that each topic would generate that specific word.(These calcuations are done interatively and I am still trying to understand how they are calculated). This process is done iteratively with the probability of a specific word for each topic being recalculated each iteration. This is performed many of times until the document reaches a steady state. 

The LDA algorithm returns a mixture of topics for each email. For example email #1, the text is split 70% between Topic Benghazi and 30% House Committee. Note that this is different from K-Means which returns disjoint clusters.

To perform the LDA topic modeling, each email will need to be transformed into a numerical dataset that the LDA algorithm can interpret. To accomplish this, all the emails will be represented in a document-term matrix. The document-term matrix is very simlar the the tf-idf matrix. From my understanding, each column in this matrix will represent an email and each row will represent a word. The value for a specific word and email combination is the frequency of that word.

Once this matrix is prepared, the **LdaModel()** function within the **gensim** package will define the number of topics I ask.

## Result
My expectation is that both methods (K-Means and LDA) will provide distinct benefits and drawbacks. K-Means will be useful in clustering emails that have strictly defined topics, for example emails that only discuss _Benghazi_ or emails that only discuss _Elections_. K-Means will be less useful in clustering emails that contain multiple topics.

On the other hand, LDA will be more useful in defining the topics for multi-topic emails. Likewise, it will be less informative in emails that a distinctly one topic.

##4. Limitations and Modeling Assumptions
1. Body of emails contains only relevant words or text specific to that email. The scraping that was performed to collect the emails was not perfect, it is very likely that some extraneous words or text slip through data cleaning.For the purposes of this project, it is assumed that all words in a document are relevant.

2. All emails are complete. Every single email was reviewed by the State Department before release, and in some cases the emails were redacted. For the purposes of this project, it is assumed that all emails are complete.

3. Emails are written using the steps outlined earlier. Without this assumption, the LDA algorithm would not be able to define K topics. This assumption includes other probabilistic assumptions on how topics are picked for an email (i.e. Dirichlet distribution).



## 5. Expected Hurdles
I anticipate two major hurdles in completing this project:

####Cleaning Data
As I have mentioned earlier, these emails were scraped from PDFs and a majority of the email body text is very dirty. I'm still reviewing many raw emails to determine what else I need to clean or edit before running through either algorithm. So far I have identified the following problems and still have no solution:

1. A handful of emails list out the schedule of the day. This is a problem as I am currently removing all numeric text from the emails. How can I retain the topic of schedule in the email without keeping the numeric values?
2. At the moment, some of the words that are extracted from the emails do not even seem to be English. How do I keep only English words?
3. Some emails contain attachments, such as word documents, that are included in the body of the email. For example, the entire text of a specific attachement will be inside the email body. I'm not sure if this is a mistake but I would like to remove this extra text.
4. If the email is part of long thread or discussion, how do I extract only the relevant text? How do I identify only the text from that specific email.
5. How can I efficiently remove email addresses and/or web addresses?

Being able to effectively clean the email body is a crucial step in ensuring that the two algorithms will return useful results. If I use dirty and muddy data with the algorithms, then I can expect to get dirty and muddy results that have litte coherence.
####Evaluating Accuracy
Since both of these algorithms are unsupervised, evaluating model accuracy is not straight-forward. When it comes to the K-Means algorithm, I understand that there are measures of similarity that can be calculated to evaluate model performance. However, I am struggling with evaluation methods for LDA topic modeling.

The only approach I can think of is calculating the LDA topic model on a training set of emails and fitting the model to test set. My concern with this approach is the small sample size of the 6,742 emails. 


## 6. Where I Need Help
1. What is the best way to determine the number of topics in LDA?
2. What is the best way to determine the number of clusters in K-Means?
3. Are there other popular topic modeling algorithms that I am not reviewing?
4. I am still searching for ways to evaluate the LDA topic modeling algorithm. Any pointers on how to evaluate this algorithm would be greatly appreciated.
5. How important is stemming to NLP algorithms?